# Student Performance Classification

## Import

In [82]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

## Set Variable

In [83]:
PIPELINE_NAME = "axelseancp-pipeline"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

In [84]:
DATA_ROOT = "data"

In [85]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Tahapan Data Ingestion

In [86]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="val", hash_buckets=2)
    ])
)

example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [87]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 28
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Tahapan Data Validation

### Membuat summary statistics

In [88]:
statistics_gen = StatisticsGen(
    examples = example_gen.outputs["examples"]
)
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 29
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [89]:
interactive_context.show(statistics_gen.outputs["statistics"])

### Membuat data schema

In [90]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 30
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [91]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Absences',INT,required,,-
'Age',INT,required,,-
'Ethnicity',INT,required,,-
'Extracurricular',INT,required,,-
'GPA',FLOAT,required,,-
'Gender',INT,required,,-
'GradeClass',FLOAT,required,,-
'Music',INT,required,,-
'ParentalEducation',INT,required,,-


### Mencari anomali pada dataset

In [92]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"]
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 31
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [93]:
interactive_context.show(example_validator.outputs["anomalies"])

## Tahapan Data Preprocessing

In [94]:
TRANSFORM_MODULE_FILE = "student_transform.py"

In [95]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft

def transformed_key(key):
    """Rename transformed key"""
    return key + "_xf"

def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features

    Args:
        inputs: map from feature keys to raw features
    
    Return:
        outputs: map from feature keys to transformed features
    
    Description:
        - apply one hot encoded to categorical features
        - apply standardization to float features and int features that isn't binary
        - apply rename of transformed features except for one hot encoded features
    """

    outputs = {}

    # One hot encode Ethnicity
    ethnicity_one_hot = tf.one_hot(tf.squeeze(inputs["Ethnicity"], axis=1), depth=4)
    outputs['Ethnicity_caucasian'] = ethnicity_one_hot[:, 0]
    outputs['Ethnicity_african_american'] = ethnicity_one_hot[:, 1]
    outputs['Ethnicity_asian'] = ethnicity_one_hot[:, 2]
    outputs['Ethnicity_other'] = ethnicity_one_hot[:, 3]

    # One hot encode Parental Education
    parent_edu_one_hot = tf.one_hot(tf.squeeze(inputs["ParentalEducation"], axis=1), depth=5)
    outputs['ParentalEducation_None'] = parent_edu_one_hot[:, 0]
    outputs['ParentalEducation_HighSchool'] = parent_edu_one_hot[:, 1]
    outputs['ParentalEducation_SomeCollege'] = parent_edu_one_hot[:, 2]
    outputs['ParentalEducation_Bachelor'] = parent_edu_one_hot[:, 3]
    outputs['ParentalEducation_Higher'] = parent_edu_one_hot[:, 4]

    # One hot encode Parental Support
    parent_support_one_hot = tf.one_hot(tf.squeeze(inputs["ParentalSupport"], axis=1), depth=5)
    outputs['ParentalSupport_None'] = parent_support_one_hot[:, 0]
    outputs['ParentalSupport_Low'] = parent_support_one_hot[:, 1]
    outputs['ParentalSupport_Moderate'] = parent_support_one_hot[:, 2]
    outputs['ParentalSupport_High'] = parent_support_one_hot[:, 3]
    outputs['ParentalSupport_VeryHigh'] = parent_support_one_hot[:, 4]

    # Standardize features
    outputs[transformed_key("Absences")] = tft.scale_to_z_score(inputs["Absences"])
    outputs[transformed_key("Age")] = tft.scale_to_z_score(inputs["Age"])
    outputs[transformed_key("GPA")] = tft.scale_to_z_score(inputs["GPA"])
    outputs[transformed_key("StudyTimeWeekly")] = tft.scale_to_z_score(inputs["StudyTimeWeekly"])
    outputs[transformed_key("GradeClass")] = tf.cast(inputs["GradeClass"], tf.int64) #Target Feature
    

    # normal features
    outputs["Extracurricular"] = inputs["Extracurricular"]
    outputs["Gender"] = inputs["Gender"]
    outputs["Music"] = inputs["Music"]
    outputs["Sports"] = inputs["Sports"]
    outputs["Tutoring"] = inputs["Tutoring"]
    outputs["Volunteering"] = inputs["Volunteering"]

    return outputs


Overwriting student_transform.py


In [96]:
transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

INFO:tensorflow:Assets written to: pipelines\axelseancp-pipeline\Transform\transform_graph\32\.temp_path\tftransform_tmp\b4c0fc0c5c9d49008eb9da4b982849cb\assets


INFO:tensorflow:Assets written to: pipelines\axelseancp-pipeline\Transform\transform_graph\32\.temp_path\tftransform_tmp\b4c0fc0c5c9d49008eb9da4b982849cb\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 32
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Tahapan Model Training & Tuning

### Membuat Trainer dan Tuner

menggabungkan modul trainer dan tuner

In [97]:
TRAINER_TUNER_MODULE_FILE = "student_trainer_tuner.py"

In [110]:
%%writefile {TRAINER_TUNER_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
import keras_tuner
from keras_tuner.engine import base_tuner
from keras_tuner import RandomSearch
import os
from tfx.components.trainer.fn_args_utils import FnArgs
from typing import NamedTuple, Dict, Text, Any

LABEL_KEY = "GradeClass"

def transformed_key(key):
    """Rename transformed key"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64)->tf.data.Dataset:
    """Get post_transform feature and create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )

    # Create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_key(LABEL_KEY)
    )

    return dataset

def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        print("Original feature_spec:", feature_spec)  # Debug print

        feature_spec.pop(LABEL_KEY)
        print("Modified feature_spec:", feature_spec)  # Debug print

        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        print("Parsed features:", parsed_features)  # Debug print

        transformed_features = model.tft_layer(parsed_features)

        # get prediction using transformed_features
        return model(transformed_features)
    
    return serve_tf_examples_fn

def get_hyperparameters() -> keras_tuner.HyperParameters:
    """Returns hyperparameters for building model"""
    hp = keras_tuner.HyperParameters()
    hp.Choice('units', [16, 32, 64], default=32)
    hp.Choice('learning_rate', [1e-2, 1e-3], default=1e-3)

    return hp

def model_builder(hparams):
    """Build machine learning model with hyperparameter tuning"""
    
    
    inputs = {
        'Age_xf': tf.keras.layers.Input(shape=(1,), name='Age_xf'),
        'Absences_xf': tf.keras.layers.Input(shape=(1,), name='Absences_xf'),
        'GPA_xf': tf.keras.layers.Input(shape=(1,), name='GPA_xf'),
        'StudyTimeWeekly_xf': tf.keras.layers.Input(shape=(1,), name='StudyTimeWeekly_xf'),
        'Ethnicity_caucasian': tf.keras.layers.Input(shape=(1,), name='Ethnicity_caucasian'),
        'Ethnicity_african_american': tf.keras.layers.Input(shape=(1,), name='Ethnicity_african_american'),
        'Ethnicity_asian': tf.keras.layers.Input(shape=(1,), name='Ethnicity_asian'),
        'Ethnicity_other': tf.keras.layers.Input(shape=(1,), name='Ethnicity_other'),
        'ParentalEducation_None': tf.keras.layers.Input(shape=(1,), name='ParentalEducation_None'),
        'ParentalEducation_HighSchool': tf.keras.layers.Input(shape=(1,), name='ParentalEducation_HighSchool'),
        'ParentalEducation_SomeCollege': tf.keras.layers.Input(shape=(1,), name='ParentalEducation_SomeCollege'),
        'ParentalEducation_Bachelor': tf.keras.layers.Input(shape=(1,), name='ParentalEducation_Bachelor'),
        'ParentalEducation_Higher': tf.keras.layers.Input(shape=(1,), name='ParentalEducation_Higher'),
        'ParentalSupport_None': tf.keras.layers.Input(shape=(1,), name='ParentalSupport_None'),
        'ParentalSupport_Low': tf.keras.layers.Input(shape=(1,), name='ParentalSupport_Low'),
        'ParentalSupport_Moderate': tf.keras.layers.Input(shape=(1,), name='ParentalSupport_Moderate'),
        'ParentalSupport_High': tf.keras.layers.Input(shape=(1,), name='ParentalSupport_High'),
        'ParentalSupport_VeryHigh': tf.keras.layers.Input(shape=(1,), name='ParentalSupport_VeryHigh'),
        'Extracurricular': tf.keras.layers.Input(shape=(1,), name='Extracurricular'),
        'Gender': tf.keras.layers.Input(shape=(1,), name='Gender'),
        'Music': tf.keras.layers.Input(shape=(1,), name='Music'),
        'Sports': tf.keras.layers.Input(shape=(1,), name='Sports'),
        'Tutoring': tf.keras.layers.Input(shape=(1,), name='Tutoring'),
        'Volunteering': tf.keras.layers.Input(shape=(1,), name='Volunteering')
    }

    concat = tf.keras.layers.Concatenate()(list(inputs.values()))

    x = layers.Dense(hparams.get('units'), activation='relu')(concat)
    x = layers.Dense(hparams.get('units'), activation='relu')(x)
    x = layers.Dense(hparams.get('units'), activation='relu')(x)
    outputs = layers.Dense(5, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.summary()

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hparams.get('learning_rate')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Tuner component will run this function

TunerFnResult = NamedTuple('TunerFnResult', [('tuner', base_tuner.BaseTuner),
                                             ('fit_kwargs', Dict[Text, Any])])

def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
    """
    Build the tuner using the KerasTuner API.
    Args:
        fn_args: Holds args as name/value pairs.
        - working_dir: working dir for tuning.
        - train_files: List of file paths containing training tf.Example data.
        - eval_files: List of file paths containing eval tf.Example data.
        - train_steps: number of train steps.
        - eval_steps: number of eval steps.
        - schema_path: optional schema of the input data.
        - transform_graph_path: optional transform graph produced by TFT.
    Returns:
        A namedtuple contains the following:
        - tuner: A BaseTuner that will be used for tuning.
        - fit_kwargs: Args to pass to tuner's run_trial function for fitting the
                        model , e.g., the training and validation dataset. Required
                        args depend on the above tuner's implementation.
    """

    # Define tuner
    tuner = RandomSearch(
        model_builder,
        objective='val_accuracy',
        max_trials=10,
        hyperparameters=get_hyperparameters(),
        directory=fn_args.working_dir,
        project_name='student_performance_classification'
    )

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)

    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            'x': train_set,
            'validation_data': val_set
        }
    )

# Trainer component will run this function
def run_fn(fn_args: FnArgs) -> None:
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )

    es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, verbose=1)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)

    hparams = keras_tuner.HyperParameters.from_config(fn_args.hyperparameters)

    # Build the model with tuned hyperparameters
    model = model_builder(hparams)

    # Train the model
    model.fit(
        x = train_set,
        validation_data = val_set,
        callbacks = [tensorboard_callback, es, mc],
        epochs=50,
        verbose=2
    )

    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
            tf.TensorSpec(
                shape=[None, 13],
                dtype=tf.string,
                name='examples'
            )
        )
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting student_trainer_tuner.py


In [99]:
from tfx.proto import trainer_pb2

### Membuat komponen Tuner

In [100]:
tuner = Tuner(
    module_file = os.path.abspath(TRAINER_TUNER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema = schema_gen.outputs['schema'],
    train_args = trainer_pb2.TrainArgs(splits=['train']),
    eval_args = trainer_pb2.EvalArgs(splits=['val'])
)

interactive_context.run(tuner)

Trial 6 Complete [00h 00m 07s]
val_accuracy: 0.8636363744735718

Best val_accuracy So Far: 0.8636363744735718
Total elapsed time: 00h 01m 03s
Results summary
Results in pipelines\axelseancp-pipeline\.temp\33\student_performance_classification
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 05 summary
Hyperparameters:
units: 64
learning_rate: 0.01
Score: 0.8636363744735718

Trial 03 summary
Hyperparameters:
units: 32
learning_rate: 0.001
Score: 0.8616600632667542

Trial 04 summary
Hyperparameters:
units: 64
learning_rate: 0.001
Score: 0.8557312488555908

Trial 01 summary
Hyperparameters:
units: 16
learning_rate: 0.01
Score: 0.8537549376487732

Trial 00 summary
Hyperparameters:
units: 16
learning_rate: 0.001
Score: 0.8438735008239746

Trial 02 summary
Hyperparameters:
units: 32
learning_rate: 0.01
Score: 0.7549406886100769


ExecutionResult(
    component_id: Tuner
    execution_id: 33
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Membuat komponen Trainer

In [111]:
trainer = Trainer(
    module_file = os.path.abspath(TRAINER_TUNER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema = schema_gen.outputs['schema'],
    hyperparameters = tuner.outputs['best_hyperparameters'],
    train_args = trainer_pb2.TrainArgs(splits=['train']),
    eval_args = trainer_pb2.EvalArgs(splits=['val'])
)

interactive_context.run(trainer)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Age_xf (InputLayer)         [(None, 1)]                  0         []                            
                                                                                                  
 Absences_xf (InputLayer)    [(None, 1)]                  0         []                            
                                                                                                  
 GPA_xf (InputLayer)         [(None, 1)]                  0         []                            
                                                                                                  
 StudyTimeWeekly_xf (InputL  [(None, 1)]                  0         []                            
 ayer)                                                                                      

INFO:tensorflow:Assets written to: pipelines\axelseancp-pipeline\Trainer\model\39\Format-Serving\assets


295/295 - 15s - loss: 0.5285 - accuracy: 0.8415 - val_loss: 0.5529 - val_accuracy: 0.8478 - 15s/epoch - 50ms/step
Epoch 2/50

Epoch 2: val_accuracy improved from 0.84783 to 0.85573, saving model to pipelines\axelseancp-pipeline\Trainer\model\39\Format-Serving
INFO:tensorflow:Assets written to: pipelines\axelseancp-pipeline\Trainer\model\39\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\axelseancp-pipeline\Trainer\model\39\Format-Serving\assets


295/295 - 8s - loss: 0.2631 - accuracy: 0.9177 - val_loss: 0.7744 - val_accuracy: 0.8557 - 8s/epoch - 26ms/step
Epoch 3/50

Epoch 3: val_accuracy did not improve from 0.85573
295/295 - 3s - loss: 0.1478 - accuracy: 0.9501 - val_loss: 1.0448 - val_accuracy: 0.8142 - 3s/epoch - 11ms/step
Epoch 4/50

Epoch 4: val_accuracy did not improve from 0.85573
295/295 - 4s - loss: 0.0851 - accuracy: 0.9726 - val_loss: 1.3609 - val_accuracy: 0.8399 - 4s/epoch - 14ms/step
Epoch 5/50

Epoch 5: val_accuracy did not improve from 0.85573
295/295 - 3s - loss: 0.0715 - accuracy: 0.9759 - val_loss: 1.5593 - val_accuracy: 0.8142 - 3s/epoch - 12ms/step
Epoch 6/50

Epoch 6: val_accuracy did not improve from 0.85573
295/295 - 4s - loss: 0.0606 - accuracy: 0.9803 - val_loss: 2.0012 - val_accuracy: 0.7885 - 4s/epoch - 14ms/step
Epoch 7/50

Epoch 7: val_accuracy did not improve from 0.85573
295/295 - 5s - loss: 0.0450 - accuracy: 0.9850 - val_loss: 2.1672 - val_accuracy: 0.8103 - 5s/epoch - 18ms/step
Epoch 8/50

E

...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ValueError: in user code:

    File "e:\Self-Learning\MLOps-Dicoding-Projects\Proyek pertama\student_trainer_tuner.py", line 52, in serve_tf_examples_fn  *
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

    ValueError: Shape must be at most rank 1 but is rank 2 for '{{node ParseExample/ParseExampleV2}} = ParseExampleV2[Tdense=[DT_INT64, DT_INT64, DT_INT64, DT_INT64, DT_FLOAT, DT_INT64, DT_INT64, DT_INT64, DT_INT64, DT_INT64, DT_FLOAT, DT_INT64, DT_INT64], dense_shapes=[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1]], num_sparse=0, ragged_split_types=[], ragged_value_types=[], sparse_types=[]](examples, ParseExample/ParseExampleV2/names, ParseExample/ParseExampleV2/sparse_keys, ParseExample/ParseExampleV2/dense_keys, ParseExample/ParseExampleV2/ragged_keys, ParseExample/Const, ParseExample/Const_1, ParseExample/Const_2, ParseExample/Const_3, ParseExample/Const_4, ParseExample/Const_5, ParseExample/Const_6, ParseExample/Const_7, Pars